In [11]:
class Board():
    def __init__(self):
        self.board = self.new()
        self.empty = ' '
    
    def new(self):
        board = [[' '] * 8 for i in range(8)]
        for i in range(8):
            for j in range(8):
                if (i+j)%2 == 0:
                    board[i][j] = '•'
        for i in range(3):
            for j in range(8):
                if (i+j)%2 == 1:
                    board[i][j] = 'O'
        for i in range(5, 8):
            for j in range(8):
                if (i+j)%2 == 1:
                    board[i][j] = 'X'
        return board
    
    def display(self):
        for i in range(len(self.board)):
            print(' ', '---+'*7 + '---')
            print(i+1, '',' | '.join(self.board[i]))
        print(' ', '---+'*7 + '---')
        print('  ', '   '.join([chr(i) for i in range(65,73)]))
        
    def is_valid_move(self, fn, fl, tn, tl, piece):
        return self.belongs(fn, fl, piece.get_piece()) and \
            self.is_empty(tn, tl) and self.forward(fn, tn, piece.get_direction()) and \
            self.dist(fn, fl, tn, tl) == 1
            
    def is_valid_jump(self, fn, fl, tn, tl, piece):
        return self.belongs(fn, fl, piece.get_piece()) and \
           self.is_empty(tn, tl) and \
           self.forward(fn, tn, piece.get_direction()) and \
           self.dist(fn, fl, tn, tl) == 2 and \
           self.jump_over_opponent(fn, fl, tn, tl, piece)
    
    def jump_over_opponent(self, fn, fl, tn, tl, piece):
        l = chr((ord(tl) + ord(fl))//2)
        return not self.is_empty(fn - piece.get_direction(), l) and \
            self.get(fn - piece.get_direction(), l) != piece.get_piece()
    
    def forward(self, f, t, direct):
        return (f - t) // abs(f - t) == direct
        
    def on_board(self, n, l):
        return n >= 0 and n <=7 and \
                ord(l) >= 65 and ord(l) <= 73
        
    def get(self, n, l):
        return self.board[n][ord(l)-65]
        
    def set_(self, n, l, piece):
        self.board[n][ord(l)-65] = piece
    
    def is_empty(self, n, l):
        return self.on_board(n, l) and self.get(n, l) == self.empty
    
    def set_empty(self, n, l):
        self.set_(n, l, self.empty)
    
    def piece(self, n, l):
        if self.on_board(n, l):
            return self.get(n, l)
        return -1
        
    # check if a given player owns a given square
    def belongs(self, n, l, player):
        return self.get(n, l) == player
    
    def dist(self, fn, fl, tn, tl):
        if abs(fn - tn) != abs(ord(fl) - ord(tl)):
            return -1
        return abs(fn - tn)
           
    def winner(self, p1, p2):
        c1, c2 = 0, 0
        for row in self.board:
            c1 += row.count(p1.get_piece())
            c2 += row.count(p1.get_piece())
        if c1 == 0:
            return p2
        elif c2 == 0:
            return p1
        return None

In [12]:
class Player():
    def __init__(self, n, board):
        if n not in [1, 2]:
            raise ValueError("Only players 1 and 2 allowed")
        self.num = n
        self.board = board
        if n == 1:
            self.piece = 'X'
            self.direction = 1
        else:
            self.piece = 'O'
            self.direction = -1
        
    def turn(self, board, fr, to):
        fn, fl = fr
        tn, tl = to
        if self.board.is_valid_move(fn, fl, tn, tl, self):
            self.board.set_(tn, tl, self.piece)
            self.board.set_empty(fn, fl)
            print("Making move:",fn, fl, '->', tn, tl)
            return True
        elif self.board.is_valid_jump(fn, fl, tn, tl, self):
            self.board.set_(tn, tl, self.piece)
            self.board.set_empty(fn-self.direction, chr((ord(tl) + ord(fl))//2))
            print("Making jump:",fn, fl, '->', tn, tl)
        else:
            print("Invalid move, try again!")
        return False
    
    def jum_possible(self, fr):
        fn, fl = fr
        tn = fn - self.direction * 2
        tl1 = chr(ord(fl) - self.direction * 2)
        tl2 = chr(ord(fl) + self.direction * 2)
        return self.board.is_valid_jump(fn, fl, tn, tl1, self) or self.board.is_valid_jump(fn, fl, tn, tl2, self)
            
    def get_direction(self):
        return self.direction
    
    def get_piece(self):
        return self.piece
    
    def name(self):
        if self.num == 1:
            return "Player One (X)"
        else:
            return "Player Two (O)"

In [13]:
class Game():
    def __init__(self):
        self.board = Board()
        self.current_player = Player(1, self.board)
        self.opponent = Player(2, self.board)
        
        
    def play(self):
        while not self.board.winner(self.current_player, self.opponent):
            self.board.display()
            inp = input(self.current_player.name() + " Please enter your move: ")
            fr, to = self.parse(inp)
            
            # if invalid input
            while fr == -1:
                inp = input(self.current_player.name() + " Please enter your move, digit first, letter second, seperated by space : ")
                fr, to = self.parse(inp)
            print("You entered " + str(fr[0]+1) + str(fr[1]) + " -> " + str(to[0]+1) + str(to[1]))
            done = self.current_player.turn(self.board, fr, to)
            
            # if other jumps available
            while not done:
                to = self.current_player.jum_possible(to)
                if to != -1:
                    fr = to
                    inp = input(self.current_player.name() + " Please enter another jump: ")
                    fr, to = self.parse(fr + ' ' + inp)
            # invalid move
            while done == -1:
                inp = input(self.current_player.name() + " Please enter your move: ")
                fr, to = self.parse(inp)
                print("You entered " + str(fr[0]+1) + str(fr[1]) + " -> " + str(to[0]+1) + str(to[1]))
                done = self.current_player.turn(self.board, fr, to)
            self.current_player, self.opponent = self.opponent, self.current_player
                
    # parse the imput to coordinates
    def parse(self, inp):
        try:
            fr, to = inp.split(' ')
            fr = (int(fr[0])-1, fr[1].upper())
            to = (int(to[0])-1, to[1].upper())
        except: 
            return -1, -1
        if fr[0] == fr[1]:
            return -1, -1
        return fr, to

In [14]:
g = Game()
g.play()

  ---+---+---+---+---+---+---+---
1  • | O | • | O | • | O | • | O
  ---+---+---+---+---+---+---+---
2  O | • | O | • | O | • | O | •
  ---+---+---+---+---+---+---+---
3  • | O | • | O | • | O | • | O
  ---+---+---+---+---+---+---+---
4    | • |   | • |   | • |   | •
  ---+---+---+---+---+---+---+---
5  • |   | • |   | • |   | • |  
  ---+---+---+---+---+---+---+---
6  X | • | X | • | X | • | X | •
  ---+---+---+---+---+---+---+---
7  • | X | • | X | • | X | • | X
  ---+---+---+---+---+---+---+---
8  X | • | X | • | X | • | X | •
  ---+---+---+---+---+---+---+---
   A   B   C   D   E   F   G   H
Player One (X) Please enter your move: 6h r6
Player One (X) Please enter your move, digit first, letter second, seperated by space : 6n 7v
You entered 6N -> 7V


IndexError: list index out of range